In [1]:
import anzoGraphClient as azg

In [2]:
graphName = '<airline_flight_network>'
azg.getTypesOfNodes(graphName)
azg.nodeTypes

['<https://ontologies.semanticarts.com/flights/Country>',
 '<https://ontologies.semanticarts.com/raw_data#Carrier>',
 '<https://ontologies.semanticarts.com/flights/City>',
 '<https://ontologies.semanticarts.com/flights/Airport>',
 '<https://ontologies.semanticarts.com/flights/ForeignCountry>',
 '<https://ontologies.semanticarts.com/flights/DomesticCountry>',
 '<https://ontologies.semanticarts.com/flights/Flight>']

In [3]:
# assign color to different node types
colors = ['#58FAF4','#d1fa58','#fada58','#fa589c','#dc58fa','#6358fa','#f7fa58']
colorCode = {
    '<https://ontologies.semanticarts.com/flights/Country>': '#fa589c',
    '<https://ontologies.semanticarts.com/raw_data#Carrier>': '#d1fa58',
    '<https://ontologies.semanticarts.com/flights/City>': '#fada58',
    '<https://ontologies.semanticarts.com/flights/Airport>': '#58FAF4',
    '<https://ontologies.semanticarts.com/flights/ForeignCountry>': '#dc58fa',
    '<https://ontologies.semanticarts.com/flights/DomesticCountry>': '#6358fa',
    '<https://ontologies.semanticarts.com/flights/Flight>': '#f7fa58'
}
azg.assignColorsToNode(colorCode)  

True

In [4]:
graphName = '<airline_flight_network>'

# Demo: construct creation function for airlines data
def createConstructQuery(origin,graphName):
    return '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix : <https://ontologies.semanticarts.com/raw_data#>
prefix fl: <https://ontologies.semanticarts.com/flights/>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
CONSTRUCT { 
  	?orig fl:hasRouteTo ?dest .
  	?orig rdfs:label ?origCode . 
  	?dest rdfs:label ?destCode . 
  	fl:hasRouteTo rdfs:label "hasRouteTo" . 
}  
WHERE { GRAPH '''+ graphName +''' {{
SELECT 
?orig ?dest ?origCode ?destCode ?miles 
    WHERE { 
        ?orig fl:terminalCode ?origCode .
        ?orig fl:hasRouteTo ?dest .
        ?dest fl:terminalCode ?destCode .
        << ?orig fl:hasRouteTo ?dest >> fl:distanceMiles ?miles .
        FILTER (?miles < 400)
        FILTER (?origCode = \'''' + origin + '''\')
  }
      }}}
'''


construct = createConstructQuery('BOS',graphName)
elements = azg.createGraph(construct,'BOS', graphName)

In [5]:
# show node label inside or outside of node
putLabelInsideNode = True

# change node size
nodeWidth = '40%'
nodeHeight = '40%'

In [6]:

from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State

app = JupyterDash(__name__)

app.layout = azg.getAppLayout(elements, nodeWidth, nodeHeight, putLabelInsideNode)


##########CALLBACKS##########

# @app.callback(Output('cytoscape-update-layout', 'elements'),
#               [Input('remove-button', 'n_clicks')],
#               [State('cytoscape-update-layout', 'elements'),
#                State('cytoscape-update-layout', 'tapNodeData')])
# def check1(x, elements, data):
#     print("Here")
#     return elements

@app.callback(Output('selected-node-data-json-output', 'children'),
              [Input('cytoscape-update-layout', 'tapNodeData')])
def displayTapNodeData(data):
    return azg.displayTapNodeData(data,graphName)


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapEdgeData')]
             )
def displayTapEdgeData(data):
    return azg.displayTapEdgeData(data)
    
@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return azg.update_layout(layout)

@app.callback(Output('cytoscape-update-layout', 'elements'),
              [Input('cytoscape-update-layout', 'tapNodeData')],
              [State('cytoscape-update-layout', 'elements'),
               State('radio-option', 'value')])
def check(data, elements, option):
    return azg.generate_elements(data,elements,option, graphName)
    
            
port = '8198'
host = 'localhost'

app.run_server(mode='inline',port = port, host = host)

print('App running on http://'+ host +':' + port +'/')

App running on http://localhost:8198/


In [7]:
# expands jupyter notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))